## Limpieza de datos: Ejercicio

### Exploración de datos

La manipulación y limpieza de datos resulta ser una tarea tediosoa, pero de suma importancia en la analítica de datos. Un buen habito siempre es tomar algunas observaciones y manipularlas manualmente para conocer más de cerca a lo que se esta haciendo frente. Puede que parezca algo aburrido o arcaico pero con seguridad pemitirá acelerar el procesamiento automático y análisis más complejos.

A continuación se presenta un caso en el que buscaremos limpiar un conjunto de datos y alistarlo para análisis más profundos

In [1]:
# Importación de librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import base64

import os

In [2]:
df = pd.read_csv("data/Limpieza/trips.csv") # Importación de datos 

En un primer momento se desa conocer el significado de cada una de las columnas, o mejor llamadas variables: A continuación se describe cada una de ellas

1. **Rental Id:** Un usuario puede alquilar cualquier bicicleta en la ciudad durante el día por un costo de £2. Esta columna corresponde al Id de una transacción de alquile, por lo que puede corresponder a más de una bicicleta (No es el Id de un viaje)

2. **Duration:** Duración del viaje en segundos

3. **Bike Id:** Identificación de la bicicleta alquilada

4. **End Date:** Momento final del viaje

5. **EndStation Id:** Id de la estación donde el viaje finalizó

6. **Start Date:** Momento de inicio del viaje

7. **StartStation** Id de la estación donde el viaje inicio

8. **tag** Etiqueta asignada a cada uno de los viajes para facilitar la agrupación de viajes

9. **userCategory** Puede se A (Usuario ocasional) o B (Usuario frecuente)

## Formato

Al momento de exportar nuestro conjunto de datos limpios va a ser necesario que nos aseguremos que el formato en que lo guardemos y su codificación sea la deseada

## Relevancia

Se ha de sustituir valores perdidos dependiendo del tipo de variable que se este procesando.

In [3]:
# Asignar el valor de "Not available" en las casillas vacías
df["Bike Id"] = df["Bike Id"].fillna('Not available')            
df["EndStation Id"] = df["EndStation Id"].fillna('Not available')

### Interpolación de valores perdidos

La interpolación consiste en asignar valores a espacios vacíos utilizando diferentes alternativas como regresiones lineales, que es el caso de la función `.iterpolate()` en pandas.

In [4]:
# Convertir la variable userCateory en una categoría y no cadena de texto

df['userCategory'] = df['userCategory'].astype('category')

In [5]:
df['userCategory'].cat.codes.unique() # El valor de -1 indica un NaN

array([ 0,  1, -1], dtype=int8)

In [6]:
#  Sustituir los valores perdidos -1 con NaN
user_cat_codes = df['userCategory'].cat.codes.replace(-1, np.nan)

# Aplicar la función de interpolación para sustituir los NaN con 0 o 1
user_cat_codes = user_cat_codes.interpolate()

In [7]:
# Convertir los códigos a valores categóricos

user_cat_codes = user_cat_codes.astype(int).astype('category')

# Reemplazar las categorías para el conjunto de datos original

df['userCategory'] = user_cat_codes

## Consistencia en el tipo de datos

Se debe verificar que el tipo de dato sea consistente con las características de las variables. Por ejemplo: `userCategory` corresponde a una variable de tipo categórica, por lo que se debe asegurar que en el DataFrame corresponda su tipo

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3767036 entries, 0 to 3767035
Data columns (total 9 columns):
 #   Column           Dtype   
---  ------           -----   
 0   Rental Id        int64   
 1   Duration         int64   
 2   Bike Id          object  
 3   End Date         object  
 4   EndStation Id    object  
 5   Start Date       object  
 6   StartStation Id  int64   
 7   tag              object  
 8   userCategory     category
dtypes: category(1), int64(3), object(5)
memory usage: 233.5+ MB


In [9]:
df["Rental Id"] = df["Rental Id"].astype("category")
df["Bike Id"] = df["Bike Id"].astype("category")
df["EndStation Id"] = df["EndStation Id"].astype("category")
df["StartStation Id"] = df["StartStation Id"].astype("category")
df["tag"] = df["tag"].astype("category")

Igualmente es preciso verificar si las categorías son consistentes para cada una de las variables. En este caso se puede evidenciar un problema de consistencia con la variable `tag`. Al parecer se les asignó el mismo nombre pero escrito de diferente manera

In [10]:
rename_dict = {"Priority low":"low",
               "priority_high":"high",
               "priority Medium":"medium",
               "priority_medium":"medium",
               "priority low":"low"
              }
df["tag"] = df["tag"].replace(rename_dict).astype("category")

## Consistencia en los nombres de las variables

Con el propósito que todas las variables estén escritas en un mismo sentido (minúsculas, sin espacios, etc), se hace una verificación y cambio en las columnas de nuestro DataFrame

In [11]:
columns_dict = {"Rental Id":"rental_id",
                "Duration":"duration",
                "Bike Id":"bike_id",
                "End Date":"end_date",
                "EndStation Id":"end_station_id",
                "Start Date":"start_date",
                "StartStation Id":"start_station_id",
                "userCategory":"user_category"
               }
df = df.rename(columns=columns_dict)

## Integridad referencial

Para garantizar que nuestro análisis sea correcto se debe verificar que no hayan valores duplicados que puedan ser redundantes y viciar el análisis. Para hacer frente a este problema pandas permite usar la función `.drop_duplicates()` que elimina aquellas observaciones donde todos los valores de las variables sean iguales

In [12]:
df = df.drop_duplicates()

In [14]:
df.columns

Index(['rental_id', 'duration', 'bike_id', 'end_date', 'end_station_id',
       'start_date', 'start_station_id', 'tag', 'user_category'],
      dtype='object')

Para los propósitos de este caso resulta que se desea construir una etiqueta para cada uno de los viajes. Cada uno de los viajes es único para la combinación: `Start Date, End Date y Bike_Id`. El módulo base 64 permite crear códigos únicos considerando una cadena de texto. A continuación se muestra un ejemplo de su uso

In [15]:
# Se crea una columna para el identificador del viaje. Su valor será la combinación de Start Date, End Date y Bike_Id
df['trip_id'] = df.apply(lambda x: ':'.join([str(x['start_date']), str(x['end_date']), str(x['bike_id'])]), axis=1)

# Codificación en base 64, decodificación y almacenamiento nuevamente en la columna trip_id
df['trip_id'] = df['trip_id'].apply(lambda x: base64.b64encode(x.encode()).decode())

df['trip_id'].unique()

array(['MDMvMDkvMjAyMCAyMDowNDowMy8wOS8yMDIwIDIwOjMyOjEyODI5LjA=',
       'MDYvMDkvMjAyMCAxMjoxMTowNi8wOS8yMDIwIDEyOjU2OjEwODYzLjA=',
       'MDIvMDkvMjAyMCAxMDo0OTowMi8wOS8yMDIwIDEwOjU2OjM5OTcuMA==', ...,
       'MDQvMDIvMjAyMSAxNToxNzowNC8wMi8yMDIxIDE1OjIyOjM5NDMuMA==',
       'MDYvMDIvMjAyMSAxMDozMzowNi8wMi8yMDIxIDEwOjM0OjIzMjcuMA==',
       'MDYvMDIvMjAyMSAxNjoxODowNi8wMi8yMDIxIDE2OjE5OjU2NS4w'],
      dtype=object)

## 3. Agragación de nueva información (Data Augmentation)

Siempre existe la posibilidad de acceder a información adicional proveniente de otros conjuntos de datos. En esta sección se usaran algunas funciones que nos permitan hacer esta agreación

### Fusión de dataframes

In [16]:
stations = pd.read_json("data/Limpieza/stations.json", orient="columns")
stations.head()

,Station ID,Latitude,Longitude,Station Name
0,1,51.529163,-0.109970,"River Street , Clerkenwell"
1,2,51.499606,-0.197574,"Phillimore Gardens, Kensington"
2,3,51.521283,-0.084605,"Christopher Street, Liverpool Street"
3,4,51.530059,-0.120973,"St. Chad's Street, King's Cross"
4,5,51.493130,-0.156876,"Sedding Street, Sloane Square"


Los archivos **JSON** son un formato similar a los diccionarios y proveniente del lenguaje de programación `JavaScript`. Su uso como alternativa a los archivos **cvs** se explica en el uso de memoria. 

El argumento de orientación indica como se orientan los datos en el archivo **JSON**.

~~~python
{
   "col 1":{
      "row 1":"This is column 1, row 1",
      "row 2":"This is column 1, row 2"
   },
   "col 2":{
      "row 1":"This is column 2, row 1",
      "row 2":"This is column 2, row 2"
   }
}
~~~

But let's say you have a file like this:

~~~json
{
   "row 1":{
      "col 1":"This is row 1, column 1",
      "col 2":"This is row 1, column 2"
   },
   "row 2":{
      "col 1":"This is row 2, column 1",
      "col 2":"This is row 2, column 2"
   }
}
~~~

Al observar la columna `location` se evidencia que no solo tienen nombre de la estación (y ubicación), por lo que sería útil si se diferencia esta información, en este caso separando los valores de las `,`. La función `str.strip()` permite manipular cadenas de texto y separarla usando un caracter específico o por espacios (en caso de considerar un único caracter se especifica con el argumento n="veces que queremos la separación")

In [17]:
split_columns = stations["Station Name"].str.split(pat=",", expand=True, n=1)
split_columns.head()

,0,1
0,River Street,Clerkenwell
1,Phillimore Gardens,Kensington
2,Christopher Street,Liverpool Street
3,St. Chad's Street,King's Cross
4,Sedding Street,Sloane Square


Uso de `pd.concat()` para unificar el arreglo fragmentado de las localizaciones  las estaciones

In [18]:
stations = pd.concat([stations, split_columns], axis=1)  #
rename_dict = {
    0:"station_name",
    1:"location",
    "Latitude":"latitude",
    "Longitude":"longitude",
    "Station ID":"station_id"
}

stations = stations.rename(columns=rename_dict)
stations = stations.drop(columns=["Station Name"])
stations["station_id"] = stations["station_id"].astype("category") # Convertir a categórico el ID de la estación

stations.head()

,station_id,latitude,longitude,station_name,location
0,1,51.529163,-0.109970,River Street,Clerkenwell
1,2,51.499606,-0.197574,Phillimore Gardens,Kensington
2,3,51.521283,-0.084605,Christopher Street,Liverpool Street
3,4,51.530059,-0.120973,St. Chad's Street,King's Cross
4,5,51.493130,-0.156876,Sedding Street,Sloane Square


In [19]:
# Asegurar que no hayan espacios vacíos al inicio o final

stations["station_name"] = stations["station_name"].str.strip()
stations["location"] = stations["location"].str.strip()

### Fusión de dataframes 

Ahora si se hace efectiva la fusión de datos de ambos DataFrames, pero ahora fusionando según los valores de las columnas

In [22]:
df = pd.merge(df, stations, left_on="start_station_id", right_on="station_id", how="left") # Fusión utilizando las columnas

rename_dict = {
    "latitude":"start_latitude",
    "longitude":"start_longitude",
    "station_name":"start_station_name",
    "location":"start_location"}

df = df.rename(columns=rename_dict)

# Eliminar las columnas adicionales redundantes
df = df.drop(columns=["station_id"])

# Completar los valores perdidos con "Not Available" - Para las columnas tipo string
obj_cols = df.columns[df.dtypes=="object"]
df[obj_cols] = df[obj_cols].fillna('Not Available')

In [23]:
# Se repite el procesimiento pero para las estaciones finales

df = pd.merge(df, stations, left_on="end_station_id", right_on="station_id", how="left") # Fusión utilizando las columnas

rename_dict = {
    "latitude":"end_latitude",
    "longitude":"end_longitude",
    "station_name":"end_station_name",
    "location":"end_location"}

df = df.rename(columns=rename_dict)

# Eliminar las columnas adicionales redundantes
df = df.drop(columns=["station_id"])

# Completar los valores perdidos con "Not Available" - Para las columnas tipo string
obj_cols = df.columns[df.dtypes=="object"]
df[obj_cols] = df[obj_cols].fillna('Not Available')

In [24]:
df.head()

,rental_id,duration,bike_id,end_date,end_station_id,start_date,start_station_id,tag,user_category,trip_id,start_latitude,start_longitude,start_station_name,start_location,end_latitude,end_longitude,end_station_name,end_location
0,101428476,1680,12829.0,03/09/2020 20:32,132.0,03/09/2020 20:04,574,medium,0,MDMvMDkvMjAyMCAyMDowNDowMy8wOS8yMDIwIDIwOjMyOj...,51.533560,-0.093150,Eagle Wharf Road,Hoxton,51.523648,-0.074754,Bethnal Green Road,Shoreditch
1,101522714,2700,10863.0,06/09/2020 12:56,702.0,06/09/2020 12:11,82,medium,1,MDYvMDkvMjAyMCAxMjoxMTowNi8wOS8yMDIwIDEyOjU2Oj...,51.514274,-0.111257,Chancery Lane,Holborn,51.528681,-0.065550,Durant Street,Bethnal Green
2,101377356,420,3997.0,02/09/2020 10:56,97.0,02/09/2020 10:49,225,medium,0,MDIvMDkvMjAyMCAxMDo0OTowMi8wOS8yMDIwIDEwOjU2Oj...,51.509353,-0.196422,Notting Hill Gate Station,Notting Hill,51.497924,-0.183834,Gloucester Road (North),Kensington
3,101393663,660,16542.0,02/09/2020 18:40,622.0,02/09/2020 18:29,97,medium,0,MDIvMDkvMjAyMCAxODoyOTowMi8wOS8yMDIwIDE4OjQwOj...,51.497924,-0.183834,Gloucester Road (North),Kensington,51.507481,-0.205535,Lansdowne Road,Ladbroke Grove
4,101622659,660,1605.0,08/09/2020 19:57,219.0,08/09/2020 19:46,36,medium,0,MDgvMDkvMjAyMCAxOTo0NjowOC8wOS8yMDIwIDE5OjU3Oj...,51.501737,-0.184980,De Vere Gardens,Kensington,51.490163,-0.190393,Bramham Gardens,Earl's Court


## Ingeniería de características

La ingeniería de características consiste en la creación de nuevas características basados en características ya existentes. Por ejemplo, el día de la semana basados en la fecha de renta de las bicicletas, hora de inicio o costo del alquiler

In [25]:
# Ya se ha generado el arreglo fusionado. Usted podría intentarlo al igual que en la sección previa
df = pd.read_csv("data/Limpieza/df_exercise.csv", parse_dates=["start_date"])

df['start_hour'] = df['start_date'].dt.hour # regresar la hora
df['start_weekday'] = df['start_date'].dt.weekday # regresar el día de la semana () como un entero

### Creación de una columna para el costo de renta

`math.ceil` redondea los valores a su entero superior

In [26]:
df['time_blocks'] = df["duration_min"].apply(lambda x: math.ceil(x/30)) # Se crean bloques de media hora (pago)

In [27]:
df['time_blocks'] = df['time_blocks'] - 1  # El primer bloque es de 2, si se demora más se le cobran 2 adicionales

In [28]:
# La costo adicional sería el producto de 2 por el número de bloques adicionales

rental_cost = df.groupby("rental_id")["time_blocks"].sum().reset_index()
rental_cost["rental_cost"] = 2 + (rental_cost["time_blocks"]*2)
rental_cost = rental_cost.drop(columns=["time_blocks"])

## Exportación de dataset

Finalmente después de contar con nuestro arreglo "limpio" procedemos a exportarlo :D

In [29]:
df.to_csv("clean.csv", encoding="utf-8")